# VacationPy
---


In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port mathurin,-19.6833,63.4167,26.83,78,2,2.63,MU,1738794945
1,1,flin flon,54.7682,-101.8650,-17.09,66,75,3.09,CA,1738794946
2,2,port-aux-francais,-49.3500,70.2167,7.39,95,99,8.84,TF,1738794921
3,3,ichnya,50.8626,32.3942,-1.24,76,100,5.21,UA,1738794949
4,4,crete,35.1558,24.8950,3.92,65,82,4.96,GR,1738794950


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
Humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    alpha = 0.5,
    color = "City",
)

# Display the map
Humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[((city_data_df["Max Temp"] > 26) & (city_data_df["Max Temp"] < 30) & (city_data_df["Humidity"] <= 60) & (city_data_df["Wind Speed"] < 5) & (city_data_df["Cloudiness"] < 30))]

# Drop any rows with null values
ideal_weather_df.dropna(how='any')

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
211,211,cabo san lucas,22.8909,-109.9124,26.17,33,0,3.60,MX,1738794924
264,264,villa corona,20.4167,-103.6833,29.16,15,0,2.69,MX,1738795271
284,284,cacule,-14.5033,-42.2222,26.86,43,21,3.31,BR,1738795295
321,321,port augusta,-32.5000,137.7667,27.22,42,0,3.53,AU,1738795341
325,325,coahuayana de hidalgo,18.7000,-103.6583,28.95,57,0,2.72,MX,1738795345
370,370,puerto baquerizo moreno,-0.9000,-89.6000,29.55,58,20,3.09,EC,1738795399
391,391,techiman,7.5862,-1.9414,27.36,15,1,0.49,GH,1738795427
513,513,ciudad lazaro cardenas,17.9583,-102.2000,28.09,57,0,3.85,MX,1738795585
533,533,san luis de la loma,17.2706,-100.8939,28.53,56,0,3.02,MX,1738795610
561,561,lazaro cardenas,17.9583,-102.2000,28.09,57,0,3.85,MX,1738795647


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy(deep = True)
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
211,cabo san lucas,MX,22.8909,-109.9124,33,
264,villa corona,MX,20.4167,-103.6833,15,
284,cacule,BR,-14.5033,-42.2222,43,
321,port augusta,AU,-32.5000,137.7667,42,
325,coahuayana de hidalgo,MX,18.7000,-103.6583,57,
370,puerto baquerizo moreno,EC,-0.9000,-89.6000,58,
391,techiman,GH,7.5862,-1.9414,15,
513,ciudad lazaro cardenas,MX,17.9583,-102.2000,57,
533,san luis de la loma,MX,17.2706,-100.8939,56,
561,lazaro cardenas,MX,17.9583,-102.2000,57,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
categories = "accommodation.hotel"
params = {
    "categories":categories, 
    "limit":limit, 
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lng = row["Lng"]
    Lat = row["Lat"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}" 

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cabo san lucas - nearest hotel: Comfort Rooms
villa corona - nearest hotel: GESA
cacule - nearest hotel: Hotel Alvorada
port augusta - nearest hotel: Hotel Commonwealth
coahuayana de hidalgo - nearest hotel: No hotel found
puerto baquerizo moreno - nearest hotel: Dory's House
techiman - nearest hotel: Encom Hotel Limited
ciudad lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
san luis de la loma - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico


,City,Country,Lat,Lng,Humidity,Hotel Name
211,cabo san lucas,MX,22.8909,-109.9124,33,Comfort Rooms
264,villa corona,MX,20.4167,-103.6833,15,GESA
284,cacule,BR,-14.5033,-42.2222,43,Hotel Alvorada
321,port augusta,AU,-32.5000,137.7667,42,Hotel Commonwealth
325,coahuayana de hidalgo,MX,18.7000,-103.6583,57,No hotel found
370,puerto baquerizo moreno,EC,-0.9000,-89.6000,58,Dory's House
391,techiman,GH,7.5862,-1.9414,15,Encom Hotel Limited
513,ciudad lazaro cardenas,MX,17.9583,-102.2000,57,Hotel Sol del Pacífico
533,san luis de la loma,MX,17.2706,-100.8939,56,No hotel found
561,lazaro cardenas,MX,17.9583,-102.2000,57,Hotel Sol del Pacífico


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
Vacation_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    alpha = 0.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
Vacation_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)